## Project 4-2 : Data Acquisition SQL

## Assignment Instructions

### Description

Some SQL problems.

### Grading

For grading purposes, we will clear all outputs from all your cells and then run them all from the top.  Please test your notebook in the same fashion before turning it in.

### Submitting Your Solution

To submit your notebook, first clear all the cells (this won't matter too much this time, but for larger data sets in the future, it will make the file smaller).  Then use the File->Download As->Notebook to obtain the notebook file.  Finally, submit the notebook file on Canvas.

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from sqlalchemy import create_engine, inspect

### Problem: SQL (50 points)

Use the csci303 database downloaded in class and code provided (12-SQL_New) to answer the following questions.  Note that you can answer every one of these questions just with single SQL queries, but you are free to instead pull data into pandas and answer the questions that way, or use multiple queries, etc.

The data you'll be working with is from the [Bureau of Labor Statistics](https://www.bls.gov/emp/documentation/education-training-system.htm).
<br>There are three tables:
- `occupation_codes` provides a mapping between two different organization's occupation codes and titles.  You'll need this table to lookup occupation codes for the other two tables.
- `occupation_attainments` gives the percentages of workers (aged 25 or older) in each occupation (in 2014-2015) who had attained a particular level of education.
- `occupation_assignments` gives requirements for each occupation in terms of education, experience, and on-the-job training.

The code to inspect the names and types of the columns in thses tables is given below. Feel free to comment this out once you get a feel for the table structures.

## SET UP

To set up we have to create an inspector on the database to get the information we need! To do so we will follow these steps:
1. Create a Database variable, this holds the name of the database you downloaded for this assignment!
2. Create an inspector object and create_engine on the database object from # 1
3. Print out information from the database to gain a better understanding of what it contains!

---

**Questions for Set Up (Edit this cell):**

---

(a) What is the variable type of the acs_code column?
The type is INTEGER.

(b) What does the 'Real' column type mean?
Real is a float function

(c) What column(s) are shared amongst the three tables? This is how they are related!!
The soc_code column is shared between all three tables. this would most likely be a Foreign Key that links to 

In [2]:
#FINISH THIS CODE

dburi = 'sqlite:///csci303.sqlite3' # Data base connection!
inspector = inspect(create_engine(dburi)) # Here we have created an inspector to take a look at the db!

tables = list(filter(lambda name: "occupation" in name, inspector.get_table_names()))

# Loop through the tables list and print all the table names and columns
for table in tables:
    print("Table name: ", table) # Print the table name (table)
    for column in inspector.get_columns(table): # Print the column name and type, think of the column as a dataframe, column['xxx']
        print("Column name: %s: %s" % (column['name'], column['type'])) # TODO enter in what you want from the columns
    print()


Table name:  occupation_assignments
Column name: soc_code: TEXT
Column name: education: TEXT
Column name: experience: TEXT
Column name: on_the_job: TEXT

Table name:  occupation_attainments
Column name: soc_code: TEXT
Column name: less_than_hs: REAL
Column name: hs_or_equivalent: REAL
Column name: some_college: REAL
Column name: associates: REAL
Column name: bachelors: REAL
Column name: masters: REAL
Column name: doctoral_or_professional: REAL

Table name:  occupation_codes
Column name: sort_order: INTEGER
Column name: soc_code: TEXT
Column name: soc_title: TEXT
Column name: acs_code: INTEGER
Column name: acs_title: TEXT



---

**Questions for Problem 2 (Edit this cell):**

---

(a) What are the SOC and ACS codes for Database administrators?
The SOC code is 15-1141 and ACS code is 1060.

(b) How many SOC occupation titles are there grouped under the ACS title 'Postsecondary teachers'?
there are 38.

(c) How much education and prior experience do you typically need to become a software developer?
about 50% had a bachelors and no background expierence.

(d) What SOC occupation has the highest percentage of doctoral or professional degree holders?
Podiatrists at 99.099998
EXTRA CREDIT: (5pts) What acs_title has the longest length? (There are two right answers!)
At 105 characters, it's: Grinding, lapping, polishing, and buffing machine tool setters, operators, and tenders, metal and plastic

In [4]:
# YOUR CODE HERE
# A
pd.read_sql_query('SELECT soc_code, soc_title, acs_code, acs_title FROM occupation_codes WHERE lower(soc_title) LIKE \'database%%\';', dburi)[:10]
# B
import sqlite3
try:
    db = sqlite3.connect('csci303.sqlite3')
    cursor = db.cursor()
except sqlite3.Error as e:
    print(e)

try:
    results = cursor.execute('SELECT soc_title FROM occupation_codes WHERE acs_title = \'Postsecondary teachers\';')
    i=0
    for x in results:
        i+= 1
    print(i)
except sqlite3.Error as e:
    print(e)
    db.rollback()

# C
query = ("SELECT * FROM occupation_assignments AS a, occupation_codes AS o ON o.soc_code = a.soc_code WHERE lower(o.soc_title) LIKE \'software %%\' OR lower(o.acs_title) LIKE \'software %%\';")


pd.read_sql_query(query, dburi)[:10]

# D
query = ("SELECT o.soc_title, MAX(a.doctoral_or_professional) FROM occupation_attainments AS a, occupation_codes AS o ON o.soc_code = a.soc_code;")

pd.read_sql_query(query, dburi)[:10]

# EC
import re

maxLen = 0
maxTitle = ''
try:
    results = cursor.execute('SELECT soc_title FROM occupation_codes;')
    for x in results:
        x = str(x)
        try:
            found = re.search("'(.+?)'", x).group(1)
        except AttributeError:
            print('you messed up')
        i = len(found)
        if i> maxLen:
            maxLen = i - 1
            maxTitle = found
except sqlite3.Error as e:
    print(e)
    db.rollback()
print(maxLen, maxTitle)

38
you messed up
105 Grinding, lapping, polishing, and buffing machine tool setters, operators, and tenders, metal and plastic 


### Questionnaire
1) How long did you spend on this assignment?
<br>2.5 - 3 hours<br>
2) What did you like about it? What did you not like about it?
<br>I thought it was really fun!<br>
3) Did you find any errors or is there anything you would like changed?
<br><br>